In [2]:
!pip install streamlit tensorflow opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00


In [3]:
import streamlit as st
import tensorflow as tf
import numpy as np
import cv2

In [4]:
# Mapping of character labels to their names
map_characters = {
    0: 'abraham_grampa_simpson', 1: 'apu_nahasapeemapetilon', 2: 'bart_simpson',
    3: 'charles_montgomery_burns', 4: 'chief_wiggum', 5: 'comic_book_guy',
    6: 'edna_krabappel', 7: 'homer_simpson', 8: 'kent_brockman',
    9: 'krusty_the_clown', 10: 'lisa_simpson', 11: 'marge_simpson',
    12: 'milhouse_van_houten', 13: 'moe_szyslak', 14: 'ned_flanders',
    15: 'nelson_muntz', 16: 'principal_skinner', 17: 'sideshow_bob'
}

In [5]:
# Load the pre-trained model
@st.cache(allow_output_mutation=True)
def load_model():
    return tf.keras.models.load_model('model.h5')

In [6]:
def preprocess_img(image):
    # Resize the image to 64x64 pixels
    resized_img = cv2.resize(image, (64, 64))
    # Reshape the image to the format the model expects: (1, 64, 64, 3)
    img_reshape = resized_img.reshape(1, 64, 64, 3)
    # Normalize the pixel values to be between 0 and 1
    img_normalized = img_reshape.astype('float32') / 255.
    return img_normalized

In [7]:
def predict_result(image):
    # Preprocess the image
    img = preprocess_img(image)
    # Use the model to make a prediction
    pred = model.predict(img)
    # Find the character with the highest prediction probability
    character = map_characters[np.argmax(pred[0])]
    # Format the character name for display
    return character.replace('_', ' ').title()

In [12]:
# Main function
def main():
    st.title("Simpsons Character Classifier")

    # Upload image widget
    uploaded_file = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])

    if uploaded_file is not None:
        # Display the uploaded image
        st.image(uploaded_file, caption='Uploaded Image.', use_column_width=True)

        # Load the model
        model = load_model()

        # Predict button
        if st.button('Predict'):
            try:
                # Read the uploaded image
                image = cv2.imdecode(np.frombuffer(uploaded_file.read(), np.uint8), cv2.IMREAD_COLOR)
                # Perform prediction
                prediction = predict_result(image)
                # Display prediction
                st.success(f"The predicted character is: {prediction}")
            except Exception as e:
                st.error(f"Error processing file: {e}")

if __name__ == "__main__":
    main()